In [78]:
# Set paths for the training data

# base_dir = 'C:\\Users\\nins_\\Desktop\\Data3\\'
base_dir = '/home/carnd/Data/'
img_dir = "IMG/"
csv_file_name = ['driving_log.csv', 'driving_log2.csv', 'driving_log3.csv', 'driving_log4.csv', 'driving_log5.csv']

In [79]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Flatten, Dense, Convolution2D, MaxPooling2D, Activation, Cropping2D, Dropout, Lambda
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
%matplotlib inline

In [80]:
# Load data from csv and images into training and label numpy arrays
df1 = pd.read_csv(base_dir+csv_file_name[0], header=None)
df2 = pd.read_csv(base_dir+csv_file_name[1], header=None)
df3 = pd.read_csv(base_dir+csv_file_name[2], header=None)
df4 = pd.read_csv(base_dir+csv_file_name[3], header=None)
df = pd.concat([df1, df2, df3, df4])

In [81]:
# Split into training and validation sets
data_train, data_valid = train_test_split(df, test_size=0.2)

In [82]:
# Generator for supplying data to the model in batches

def data_generator(data, batch_size, angle_bias=0.25):
    
    data = np.array(data)
    total_samples = len(data)
    
    while(True):
        
        # Shuffle the data before each epoch
        data = shuffle(data)
        
        # Obtain samples equal to batch size and process the images
        for offset in range(0, total_samples, batch_size):
            batch = data[offset:batch_size+offset]
            
            batch_images = []
            batch_labels = []
            
            # Load images and labels equal to the batch size only
            for row in batch:
                
                img_name = row[0].split("\\")[-1]
                img_C = cv2.cvtColor(cv2.imread(base_dir+img_dir+img_name), cv2.COLOR_BGR2RGB)
                label_C = float(row[3])
                
                img_name = row[1].split("\\")[-1]
                img_L = cv2.cvtColor(cv2.imread(base_dir+img_dir+img_name), cv2.COLOR_BGR2RGB)
                label_L = label_C + angle_bias
                
                img_name = row[2].split("\\")[-1]
                img_R = cv2.cvtColor(cv2.imread(base_dir+img_dir+img_name), cv2.COLOR_BGR2RGB)
                label_R = label_C - angle_bias
                
                # Append to the batch data
                batch_images.append(img_C)
                batch_images.append(img_L)
                batch_images.append(img_R)
                
                batch_labels.append(label_C)
                batch_labels.append(label_L)
                batch_labels.append(label_R)
                
                # Flip the image
                flip_img_C = cv2.flip(img_C, 1)
                flip_label_C = label_C * (-1)
                
                flip_img_L = cv2.flip(img_L, 1)
                flip_label_L = label_L * (-1)
                
                flip_img_R = cv2.flip(img_R, 1)
                flip_label_R = label_R * (-1)
                
                # Append the augmented data to the batch data
                batch_images.append(flip_img_C)
                batch_images.append(flip_img_L)
                batch_images.append(flip_img_R)
                
                batch_labels.append(flip_label_C)
                batch_labels.append(flip_label_L)
                batch_labels.append(flip_label_R)
                
                          
            X = np.array(batch_images)
            y = np.array(batch_labels)

            yield shuffle(X, y)

In [83]:
# Create generators for training and validation

train_generator = data_generator(data_train, batch_size=32)
valid_generator = data_generator(data_valid, batch_size=32)

In [84]:
# Model Definition

model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,20), (0,0))))

model.add(Convolution2D(8, 5, 5, border_mode='valid', subsample=(1,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation(activation='relu'))

model.add(Convolution2D(6, 3, 3, border_mode='valid', subsample=(1,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation(activation='relu'))

model.add(Convolution2D(6, 3, 3, border_mode='same', subsample=(1,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation(activation='relu'))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

adam_opt = Adam(lr=0.005)
model.compile(loss='mse', optimizer=adam_opt)
filepath="model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True)
earlystop = EarlyStopping(monitor='val_loss', patience=3)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_7 (Lambda)                (None, 160, 320, 3)   0           lambda_input_7[0][0]             
____________________________________________________________________________________________________
cropping2d_12 (Cropping2D)       (None, 90, 320, 3)    0           lambda_7[0][0]                   
____________________________________________________________________________________________________
convolution2d_34 (Convolution2D) (None, 86, 316, 8)    608         cropping2d_12[0][0]              
____________________________________________________________________________________________________
maxpooling2d_34 (MaxPooling2D)   (None, 43, 158, 8)    0           convolution2d_34[0][0]           
___________________________________________________________________________________________

In [85]:
# Train the model

history = model.fit_generator(train_generator, callbacks=[checkpoint, earlystop], samples_per_epoch=len(data_train), validation_data=valid_generator, nb_val_samples=len(data_valid), nb_epoch=40)

Epoch 1/40
4608/4670 [============================>.] - ETA: 0s - loss: 0.1109

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


4800/4670 [==============================] - 11s - loss: 0.1087 - val_loss: 0.0647
Epoch 2/40
4800/4670 [==============================] - 10s - loss: 0.0577 - val_loss: 0.0650
Epoch 3/40
4800/4670 [==============================] - 10s - loss: 0.0584 - val_loss: 0.0616
Epoch 4/40
4800/4670 [==============================] - 10s - loss: 0.0571 - val_loss: 0.0589
Epoch 5/40
4800/4670 [==============================] - 10s - loss: 0.0652 - val_loss: 0.0667
Epoch 6/40
4788/4670 [==============================] - 10s - loss: 0.0550 - val_loss: 0.0586
Epoch 7/40
4800/4670 [==============================] - 10s - loss: 0.0547 - val_loss: 0.0474
Epoch 8/40
4800/4670 [==============================] - 10s - loss: 0.0486 - val_loss: 0.0330
Epoch 9/40
4800/4670 [==============================] - 10s - loss: 0.0418 - val_loss: 0.0353
Epoch 10/40
4800/4670 [==============================] - 10s - loss: 0.0367 - val_loss: 0.0377
Epoch 11/40
4800/4670 [==============================] - 10s - loss: 0